In [15]:
import requests
import json
import pandas as pd
import numpy as np
from pymongo import MongoClient
import math
from bs4 import BeautifulSoup
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster
#import src.functions as fn

In [2]:
client = MongoClient("mongodb://localhost/companies")
db = client.get_database()

## Indice calidad de vida

En primer lugar, voy a basarme en el índice de calidad de vida para seleccionar una zona. 

El índice de calidad de vida se calcula en base otros índices: índice de poder adquisitivo, índice de seguridad, indice de sanidad, de costo de visa de tiempo de desplazamiento den tráfico, contaminación clima y relación precio/ingresos para propiedades.

Para obtener esta información he hecho scrapping de la web https://es.numbeo.com/calidad-de-vida/clasificaciones-por-pa%C3%ADs.

In [ ]:
data=requests.get("https://es.numbeo.com/calidad-de-vida/clasificaciones-por-pa%C3%ADs").text
soup= BeautifulSoup(data, 'html.parser')

def procesaIndices(fila):
    m = fila.find_all("td")
    #print(m[2])
    return {
        "country":m[1].text.strip(),
        "calidad_vida":float((m[2].text).replace(',','.')),
        "poder_adquisitivo":float((m[3].text).replace(',','.')),
        "seguridad":float((m[4].text).replace(',','.')),
        "sanidad":float((m[5].text).replace(',','.')),
        "costo_vida":float((m[6].text).replace(',','.')),
        "relacion_precio_vs_ingresos":float((m[7].text).replace(',','.')),
        "tiempo_desplazamiento":float((m[8].text).replace(',','.')),
        "contaminación":float((m[9].text).replace(',','.')),
        "clima":float((m[10].text).replace(',','.'))
    }
    
indice = soup.find_all('table')[2]
indice_dict = [procesaIndices(fila) for fila in indice.find_all("tr")[1:]]


In [ ]:
#Genero df:
df_indices=pd.DataFrame(indice_dict)

#Exporto en formato json y luego importo en MongoDB Compass:
df_indices.to_json("../output/indices_calidad.json", orient="records")

El país que elegiré para poner la nueva sede, será aquel con mejor indice de calidad y además con un indice de clima superior a 90. Como se ve a continuación será Australia.

In [ ]:
df_indices[(df_indices.clima>90)].head()

## Importo de MongoDB Compass:
Con una query me traeré de la colección 'companies' todas las empresas que tenga una ganancia distinta de 0$.

In [3]:
q1={"total_money_raised": {"$ne": "$0"}}
companies=list(db["companies"].find(q1,{"name":1,"founded_year":1,"total_money_raised":1, "offices":1,"category_code":1}))

In [4]:
df = pd.DataFrame(companies)

Voy a tratar el df para desagregar la columna offices, dejando cada oficina en una línea distina y luego desdoblando las columnas que contienen la información de cada oficina.

In [5]:
#Desagrego las oficinas, para que por compañía cada oficina aparezca en una línea independiente:
df = df.explode('offices')
display(df.head())

#Desagrego la columna office para obtener los datos en columnas de cada oficina:
df_offices = df[["offices"]].apply(lambda x: x.offices, result_type="expand", axis=1)
display(df_offices.head())

#Unifico los dos df anteriores para tener un único df completo con toda la información
clean_data = pd.concat([df,df_offices], axis=1)
display(clean_data.head())

,_id,name,category_code,founded_year,total_money_raised,offices
0,52cdef7c4bab8bd675297d8a,Wetpaint,web,2005.0,$39.8M,"{'description': '', 'address1': '710 - 2nd Ave..."
0,52cdef7c4bab8bd675297d8a,Wetpaint,web,2005.0,$39.8M,"{'description': '', 'address1': '270 Lafayette..."
1,52cdef7c4bab8bd675297d8f,Omnidrive,network_hosting,2005.0,$800k,"{'description': '', 'address1': 'Suite 200', '..."
2,52cdef7c4bab8bd675297d8d,Digg,news,2004.0,$45M,"{'description': None, 'address1': '135 Mississ..."
3,52cdef7c4bab8bd675297d91,Geni,web,2006.0,$16.5M,"{'description': 'Headquarters', 'address1': '9..."


,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude
0,,710 - 2nd Avenue,Suite 1100,98104,Seattle,WA,USA,47.603122,-122.333253
0,,270 Lafayette Street,Suite 505,10012,New York,NY,USA,40.723731,-73.996431
1,,Suite 200,654 High Street,94301,Palo Alto,CA,ISR,NaN,NaN
2,None,135 Mississippi St,,94107,San Francisco,CA,USA,37.764726,-122.394523
3,Headquarters,9229 W. Sunset Blvd.,,90069,West Hollywood,CA,USA,34.090368,-118.393064


,_id,name,category_code,founded_year,total_money_raised,offices,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude
0,52cdef7c4bab8bd675297d8a,Wetpaint,web,2005.0,$39.8M,"{'description': '', 'address1': '710 - 2nd Ave...",,710 - 2nd Avenue,Suite 1100,98104,Seattle,WA,USA,47.603122,-122.333253
0,52cdef7c4bab8bd675297d8a,Wetpaint,web,2005.0,$39.8M,"{'description': '', 'address1': '270 Lafayette...",,270 Lafayette Street,Suite 505,10012,New York,NY,USA,40.723731,-73.996431
1,52cdef7c4bab8bd675297d8f,Omnidrive,network_hosting,2005.0,$800k,"{'description': '', 'address1': 'Suite 200', '...",,Suite 200,654 High Street,94301,Palo Alto,CA,ISR,NaN,NaN
2,52cdef7c4bab8bd675297d8d,Digg,news,2004.0,$45M,"{'description': None, 'address1': '135 Mississ...",None,135 Mississippi St,,94107,San Francisco,CA,USA,37.764726,-122.394523
3,52cdef7c4bab8bd675297d91,Geni,web,2006.0,$16.5M,"{'description': 'Headquarters', 'address1': '9...",Headquarters,9229 W. Sunset Blvd.,,90069,West Hollywood,CA,USA,34.090368,-118.393064


Elimino la columna _id, ya que se generará un nuevo id cuando cargue la nueva colección, y también elimino office que ya está toda la información desagregada y no nos sirve más.

In [6]:
clean_data = clean_data.drop(columns=["_id","offices"])
clean_data.head()

,name,category_code,founded_year,total_money_raised,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude
0,Wetpaint,web,2005.0,$39.8M,,710 - 2nd Avenue,Suite 1100,98104,Seattle,WA,USA,47.603122,-122.333253
0,Wetpaint,web,2005.0,$39.8M,,270 Lafayette Street,Suite 505,10012,New York,NY,USA,40.723731,-73.996431
1,Omnidrive,network_hosting,2005.0,$800k,,Suite 200,654 High Street,94301,Palo Alto,CA,ISR,NaN,NaN
2,Digg,news,2004.0,$45M,None,135 Mississippi St,,94107,San Francisco,CA,USA,37.764726,-122.394523
3,Geni,web,2006.0,$16.5M,Headquarters,9229 W. Sunset Blvd.,,90069,West Hollywood,CA,USA,34.090368,-118.393064


Con la siguiente función, daré el formato GeoJson, necesario para poder generar los indexes en MongoDB Compass. En aquellos casos en los que latitud o longitud sean valores NaN, se indicará None, para evitar posibles conflicos en la carga de la colección.

In [7]:
def asGeoJSON(lat,lng):
    try:
        lat = float(lat)
        lng = float(lng)
        if not math.isnan(lat) and not math.isnan(lng):
            return {
                "type":"Point",
                "coordinates":[lng,lat]
            }
    except Exception:
        print("Invalid data")
        return None
        

clean_data["location"] = clean_data[["latitude","longitude"]].apply(lambda x:asGeoJSON(x.latitude,x.longitude), axis=1)
clean_data[["latitude","longitude","location"]].head()

,latitude,longitude,location
0,47.603122,-122.333253,"{'type': 'Point', 'coordinates': [-122.333253,..."
0,40.723731,-73.996431,"{'type': 'Point', 'coordinates': [-73.9964312,..."
1,NaN,NaN,None
2,37.764726,-122.394523,"{'type': 'Point', 'coordinates': [-122.394523,..."
3,34.090368,-118.393064,"{'type': 'Point', 'coordinates': [-118.393064,..."


Añado una columna que tipifique si la empresa tiene 10 o menos años('young') o más de 10 años ('old').

In [8]:
def oldYoung(row):
    if row['founded_year']>=2007:
        return 'young'
    else:
        return 'old'

clean_data['old_young']= clean_data.apply(lambda row: oldYoung(row), axis=1)

clean_data[['name', 'founded_year','old_young']].head(10)

,name,founded_year,old_young
0,Wetpaint,2005.0,old
0,Wetpaint,2005.0,old
1,Omnidrive,2005.0,old
2,Digg,2004.0,old
3,Geni,2006.0,old
4,Gizmoz,2003.0,old
5,StumbleUpon,2002.0,old
5,StumbleUpon,2002.0,old
6,Slacker,2006.0,old
7,Scribd,2007.0,young


Exporto a json y luego lo importo a MongoDB Compass.

In [10]:
clean_data.to_json("output/cleaned_offices.json", orient="records")

Lo importo en Mongo DB Compass: $ mongoimport --db companies --collection offices --jsonArray --drop cleaned_offices.json

Una vez importado, creo un geospartial index: Indexes > Create Index > (write any indexname) > Select fieldname:"location" and 2dsphere

## Exporto de MongoDB:
Ahora exporto las oficinas que están en Australia para ver en un mapa su distribución

In [11]:
q2={"$and": [{"country_code": {"$eq": "AUS"}}, {"location": {"$ne":None}}]}
aus = list(db["offices"].find(q2, {'_id':0,'name':1, 'total_money_raised':1,'category_code':1, 'founded_year':1, 'city':1, 'latitude':1, 'longitude':1, 'old_young':1}))
df_aus = pd.DataFrame(aus)
df_aus.head()

,name,category_code,founded_year,total_money_raised,city,latitude,longitude,old_young
0,Google,search,1998.0,$555M,Melbourne,-37.879234,145.073608,old
1,Google,search,1998.0,$555M,Sydney,-34.822723,138.612396,old
2,Hitwise,web,NaN,$4.2M,Melbourne,-37.814251,144.963169,old
3,2threads,games_video,2004.0,$300k,sydney,-33.884685,151.216427,old
4,99designs,design,2008.0,$35M,Collingwood,-37.802659,144.986855,young


## Mapa en folium:

Antes consultaré las coordenadas de Australia en geoCode con la siguiente función:

In [13]:
def geocode(address):
    data = requests.get(f"https://geocode.xyz/{address}?json=1").json()
    print(data)
    return {
        "type":"Point",
        "coordinates":[float(data["longt"]),float(data["latt"])]
    }

aus_geo = geocode("Melbourn")

{'standard': {'addresst': {}, 'city': 'Melbourn', 'prov': 'UK', 'countryname': 'United Kingdom', 'postal': {}, 'confidence': '0.90'}, 'longt': '0.02860', 'alt': {'loc': {'longt': '0.01950', 'prov': 'UK', 'city': 'Melbourn', 'countryname': 'United Kingdom', 'postal': 'SG8', 'region': {}, 'latt': '52.08366'}}, 'elevation': {}, 'latt': '52.09674'}


In [18]:
start_lat = -25.2743988
start_lon = 133.7751312
aus_map = folium.Map(location=[start_lat, start_lon],tiles='cartodbpositron', zoom_start=4)
aus_map

In [19]:
df_aus.apply(lambda row:folium.Marker(location=[row["latitude"], 
                                                row["longitude"]], popup=row["old_young"]).add_to(aus_map),axis=1)
aus_map

Genero query para traeme los aeropuertos que hay en Australia

In [20]:
q3={"$and": [{"country": {"$eq":"Australia"}}, {"$or":[{"city":{"$eq":"Sydney"}},{"city":{"$eq":"Melbourne"}}]}]}
aus_airport = list(db['airports'].find(q3,{"_id":0, "name":1,"city":1,"lat":1,"lon":1}))

df_airports = pd.DataFrame (aus_airport)
df_airports.head()

,lat,lon,name,city
0,-37.6759,144.844,Melbourne International Airport,Melbourne
1,-33.9344,151.168,Kingsford Smith Airport,Sydney
2,-33.9344,151.168,Kingsford Smith Airport,Sydney


In [21]:
df_airports.apply(lambda row:folium.Marker(location=[row["lat"], 
                                                row["lon"]], popup=row["name"],icon=folium.Icon(color='black', icon_color='white', icon='plane')).add_to(aus_map),axis=1)
aus_map

## Colsultas a GooglePlace: starbucks, pubs, parques...

Request a la API de google consultando los starbucks en sydney.

In [47]:
'''import os
from dotenv import load_dotenv
load_dotenv('src/.env')

def requestGooglePlace(place):
    token = os.getenv("API_GOOGLE_KEY")
    #print(token)
    if not token:
        raise ValueError("Necesitas un API_GOOGLE_KEY")
    
    baseUrl = "https://maps.googleapis.com/"
    endpoint="maps/api/place/textsearch/json"
    url = baseUrl+endpoint
    print(f"Requesting data from {url}")
    params = {
        "query":f"{place}",
        "key": f"{token}"
    }
    res = requests.get(url,params=params)
    if res.status_code != 200:
        print(res.text)
        raise ValueError("Bad Response")
    return res.json()'''

#Prueba: data_Starbucks=requestGooglePlace("Starbucks+in+Sydney")

## Request de Starbucks en Melbourn

Elijo Melbourn como ciudad para establecer la oficina, ya que hay más empresas jóvenes que en Sydney.

In [45]:
#data_Starbucks_Melbourn=requestGooglePlace("Starbucks+in+Melbourn+Australia")

Requesting data from https://maps.googleapis.com/maps/api/place/textsearch/json?query=Starbucks+in+Melbourn+Australia


Extraigo la key "results" y convierto a dataframe para trabajar con la columna "geometry"

In [83]:
results_mel=data_Starbucks_Melbourn['results']
df_starbucks_melbourn = pd.DataFrame(results_mel)

#Guardo el response de la request lanzada:
json_starbucks_melbourn=json.dumps(results_mel)
writeFile =open('starbucks_melbourn.json', 'w')
writeFile.write(json_starbucks_melbourn)
writeFile.close()

In [84]:
#Desagrego la columna geometry.location para obtener la latitud y longitud en columnas de cada Starbucks:
df_loc_mel_cafe = df_starbucks_melbourn[["geometry"]].apply(lambda x: x.geometry, result_type="expand", axis=1)

df_loc_mel_cafe2 = df_loc_mel_cafe[["location"]].apply(lambda x: x.location, result_type="expand", axis=1)
display(df_loc_syd_cafe2.head())

#Unifico los dos df anteriores para tener un único df completo con toda la información
clean_starbucks_melbourn = pd.concat([df_starbucks_melbourn,df_loc_mel_cafe2], axis=1)
display(clean_starbucks_melbourn.head())

,lat,lng
0,-33.875464,151.206297
1,-33.872376,151.206620
2,-33.865223,151.205477
3,-33.766014,150.817496
4,-33.879241,151.205680


,formatted_address,geometry,icon,id,name,opening_hours,photos,place_id,plus_code,price_level,rating,reference,types,user_ratings_total,lat,lng
0,"295 Swanston St, Melbourne VIC 3000, Australia","{'location': {'lat': -37.8113086, 'lng': 144.9...",https://maps.gstatic.com/mapfiles/place_api/ic...,428668de1bda3a762d61c901dfdaf78a5ae6a559,Starbucks,{'open_now': True},"[{'height': 1920, 'html_attributions': ['<a hr...",ChIJgfh7WspC1moRCKdd0o5WEy8,"{'compound_code': '5XQ7+FP Melbourne, Victoria...",2,3.9,ChIJgfh7WspC1moRCKdd0o5WEy8,"[cafe, food, point_of_interest, store, establi...",948,-37.811309,144.964310
1,"Tivoli Arcade, 235-251 Bourke St, Melbourne VI...","{'location': {'lat': -37.8134938, 'lng': 144.9...",https://maps.gstatic.com/mapfiles/place_api/ic...,1ca754c4e5b67f2463b63dc33797c46b31915538,Starbucks,{'open_now': True},"[{'height': 3024, 'html_attributions': ['<a hr...",ChIJ8TrsBcpC1moRJLQvmKZ1_6k,"{'compound_code': '5XP8+JG Melbourne, Victoria...",2,3.9,ChIJ8TrsBcpC1moRJLQvmKZ1_6k,"[cafe, food, point_of_interest, store, establi...",839,-37.813494,144.966352
2,"Spencer Outlet Centre, 201 Spencer St, Melbour...","{'location': {'lat': -37.8161662, 'lng': 144.9...",https://maps.gstatic.com/mapfiles/place_api/ic...,1251ab8f72fb0d4edaaa27a920bf4d1a4e1503d9,Starbucks,{'open_now': False},"[{'height': 4032, 'html_attributions': ['<a hr...",ChIJYx8bFVBd1moRMTl1DA1h5mk,"{'compound_code': '5XM2+GR Melbourne, Victoria...",2,3.7,ChIJYx8bFVBd1moRMTl1DA1h5mk,"[cafe, food, point_of_interest, store, establi...",573,-37.816166,144.952079
3,"2 Elizabeth St, Melbourne VIC 3004, Australia","{'location': {'lat': -37.8178938, 'lng': 144.9...",https://maps.gstatic.com/mapfiles/place_api/ic...,173a0b592f6846db854f2f13d37cdeed18b991bf,Starbucks,{'open_now': False},"[{'height': 4032, 'html_attributions': ['<a hr...",ChIJt44AHw1D1moRipc46Fz7uzA,"{'compound_code': '5XJ8+R2 Melbourne, Victoria...",2,3.6,ChIJt44AHw1D1moRipc46Fz7uzA,"[cafe, food, point_of_interest, store, establi...",165,-37.817894,144.965043
4,"385 Bourke St, Melbourne VIC 3000, Australia","{'location': {'lat': -37.8148528, 'lng': 144.9...",https://maps.gstatic.com/mapfiles/place_api/ic...,a7da2b94c960fccf004e244f7caeadf254528be7,Starbucks Galleria,{'open_now': False},"[{'height': 4032, 'html_attributions': ['<a hr...",ChIJ_f021h1D1moRABbMDej5Hjw,"{'compound_code': '5XP7+35 Melbourne, Victoria...",2,3.7,ChIJ_f021h1D1moRABbMDej5Hjw,"[cafe, food, point_of_interest, establishment]",93,-37.814853,144.962910


Añado columna con formato geoquery:

In [89]:
clean_starbucks_melbourn["location"] = clean_starbucks_melbourn[["lat","lng"]].apply(lambda x:asGeoJSON(x.lat,x.lng), axis=1)
new_clean_starbucks_melbourn = clean_starbucks_melbourn[["name","lat","lng","location"]]
new_clean_starbucks_melbourn.head()

,name,lat,lng,location
0,Starbucks,-37.811309,144.964310,"{'type': 'Point', 'coordinates': [144.96431, -..."
1,Starbucks,-37.813494,144.966352,"{'type': 'Point', 'coordinates': [144.9663518,..."
2,Starbucks,-37.816166,144.952079,"{'type': 'Point', 'coordinates': [144.9520791,..."
3,Starbucks,-37.817894,144.965043,"{'type': 'Point', 'coordinates': [144.965043, ..."
4,Starbucks Galleria,-37.814853,144.962910,"{'type': 'Point', 'coordinates': [144.96291, -..."


In [90]:
clean_starbucks_melbourn.to_json("output/new_clean_starbucks_melbourn.json", orient="records")

In [99]:
clean_starbucks_melbourn.apply(lambda row:folium.Marker(location=[row["lat"], 
                                                row["lng"]], popup=row["name"],icon=folium.Icon(color='darkgreen', icon_color='white',icon='S')).add_to(aus_map),axis=1)
aus_map

## Request Pubs en Melbourn

In [57]:
#data_pubs=requestGooglePlace("pubs+in+Melbourn+Australia")

Requesting data from https://maps.googleapis.com/maps/api/place/textsearch/json


In [92]:
results_pubs_melbourn=data_pubs['results']
df_pubs_melbourn = pd.DataFrame(results_pubs_melbourn)
json_pubs_melbourn=json.dumps(results_pubs_melbourn)

writeFile =open('output/pubs_melbourn.json', 'w')
writeFile.write(json_pubs_melbourn)
writeFile.close()

In [93]:
#Desagrego la columna geometry.location para obtener la latitud y longitud en columnas de cada Starbucks:
df_loc_mel_pubs = df_pubs_melbourn[["geometry"]].apply(lambda x: x.geometry, result_type="expand", axis=1)

df_loc_mel_pubs2 = df_loc_mel_pubs[["location"]].apply(lambda x: x.location, result_type="expand", axis=1)
display(df_loc_mel_pubs2.head())

#Unifico los dos df anteriores para tener un único df completo con toda la información
clean_pubs_melbourn = pd.concat([df_pubs_melbourn,df_loc_mel_pubs2], axis=1)
display(clean_pubs_melbourn.head())

,lat,lng
0,-37.813709,144.968264
1,-37.816221,144.960498
2,-37.812854,144.971338
3,-37.817364,144.960836
4,-37.810597,144.962707


,formatted_address,geometry,icon,id,name,opening_hours,photos,place_id,plus_code,price_level,rating,reference,types,user_ratings_total,lat,lng
0,"127 Russell St, Melbourne VIC 3000, Australia","{'location': {'lat': -37.813709, 'lng': 144.96...",https://maps.gstatic.com/mapfiles/place_api/ic...,86a2eea528177996e0cf33a3fa8ea23fa4c927e4,The Crafty Squire,{'open_now': True},"[{'height': 3024, 'html_attributions': ['<a hr...",ChIJazndwslC1moRIfHbO-8g0ug,"{'compound_code': '5XP9+G8 Melbourne, Victoria...",2.0,4.2,ChIJazndwslC1moRIfHbO-8g0ug,"[bar, restaurant, food, point_of_interest, est...",2474,-37.813709,144.968264
1,"427 Little Collins St, Melbourne VIC 3000, Aus...","{'location': {'lat': -37.8162205, 'lng': 144.9...",https://maps.gstatic.com/mapfiles/place_api/ic...,725ad1f0351293cf67c58f216fc5aeb5de20e1a4,The Irish Times Pub,{'open_now': True},"[{'height': 3040, 'html_attributions': ['<a hr...",ChIJidRmV0td1moR9NIteGY9FmM,"{'compound_code': '5XM6+G5 Melbourne, Victoria...",2.0,4.3,ChIJidRmV0td1moR9NIteGY9FmM,"[bar, restaurant, food, point_of_interest, est...",1105,-37.816221,144.960498
2,"68 Little Collins St, Melbourne VIC 3000, Aust...","{'location': {'lat': -37.8128542, 'lng': 144.9...",https://maps.gstatic.com/mapfiles/place_api/ic...,0878d2752318253065126edf8f5077df4cdee16f,Cherry,{'open_now': False},"[{'height': 4032, 'html_attributions': ['<a hr...",ChIJk09g4LdC1moRFzfxUe2lPQM,"{'compound_code': '5XPC+VG Melbourne, Victoria...",1.0,4.3,ChIJk09g4LdC1moRFzfxUe2lPQM,"[bar, point_of_interest, establishment]",690,-37.812854,144.971338
3,"415 Collins St, Melbourne VIC 3000, Australia","{'location': {'lat': -37.8173637, 'lng': 144.9...",https://maps.gstatic.com/mapfiles/place_api/ic...,a8bbce5a30ebbea329c5df580d9f4701282c9ae0,The Sherlock Holmes,{'open_now': True},"[{'height': 1200, 'html_attributions': ['<a hr...",ChIJNXEktExd1moRpSaUftMdjdQ,"{'compound_code': '5XM6+38 Melbourne, Victoria...",2.0,4.3,ChIJNXEktExd1moRpSaUftMdjdQ,"[bar, restaurant, food, point_of_interest, est...",1032,-37.817364,144.960836
4,"211 La Trobe St, Melbourne VIC 3000, Australia","{'location': {'lat': -37.8105973, 'lng': 144.9...",https://maps.gstatic.com/mapfiles/place_api/ic...,08865cadeaf08ce26e882e5c76167b6ebb981c8c,Melbourne Central Lion Hotel,{'open_now': True},"[{'height': 4000, 'html_attributions': ['<a hr...",ChIJu-bfCMtC1moREvQhzz-vdSg,"{'compound_code': '5XQ7+Q3 Melbourne, Victoria...",2.0,3.9,ChIJu-bfCMtC1moREvQhzz-vdSg,"[bar, point_of_interest, establishment]",1368,-37.810597,144.962707


In [94]:
clean_pubs_melbourn["location"] = clean_pubs_melbourn[["lat","lng"]].apply(lambda x:asGeoJSON(x.lat,x.lng), axis=1)
new_clean_pubs_melbourn = clean_pubs_melbourn[["name","lat","lng","location"]]
new_clean_pubs_melbourn.head()

,name,lat,lng,location
0,The Crafty Squire,-37.813709,144.968264,"{'type': 'Point', 'coordinates': [144.968264, ..."
1,The Irish Times Pub,-37.816221,144.960498,"{'type': 'Point', 'coordinates': [144.9604985,..."
2,Cherry,-37.812854,144.971338,"{'type': 'Point', 'coordinates': [144.9713379,..."
3,The Sherlock Holmes,-37.817364,144.960836,"{'type': 'Point', 'coordinates': [144.9608363,..."
4,Melbourne Central Lion Hotel,-37.810597,144.962707,"{'type': 'Point', 'coordinates': [144.9627075,..."


In [95]:
clean_pubs_melbourn.to_json("output/new_clean_pubs_melbourn.json", orient="records")

In [97]:
clean_pubs_melbourn.apply(lambda row:folium.Marker(location=[row["lat"], 
                                                row["lng"]], popup=row["name"],icon=folium.Icon(color='pink', icon_color='white', icon='glass')).add_to(aus_map),axis=1)
aus_map